In [2]:
#Libraries needed
import requests
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

/Users/sahilwadhwa/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sahilwadhwa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Libraries needed part 2
from collections import Counter
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sahilwadhwa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/sahilwadhwa/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
#Libraries need part 3
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.cluster import adjusted_rand_score

In [7]:
df_combined = pd.read_csv('combined_job_data.csv')
df_combined.head()

,title,company,location,description,role,clean_description,tokens
0,Product Software Engineer,Qualcomm,"San Diego, CA","Company:\nQualcomm Innovation Center, Inc.\n\n...",software engineer,company qualcomm innovation center inc job are...,"['company', 'qualcomm', 'innovation', 'center'..."
1,Software Engineer–DevSecOps (Virtual),Boeing,"Long Beach, CA (+2 others)","Job Description\n\nAt Boeing, we innovate and ...",software engineer,job description boeing innovate collaborate ma...,"['job', 'description', 'boeing', 'innovate', '..."
2,Software Engineer I - iOS,Lensa,"Irvine, CA",Lensa is a U.S. career site that helps job see...,software engineer,lensa us career site helps job seekers discove...,"['lensa', 'us', 'career', 'site', 'helps', 'jo..."
3,DevSecOps Software Engineers (Associate/Experi...,Boeing,"El Segundo, CA (+1 other)","Job Description\n\nAt Boeing, we innovate and ...",software engineer,job description boeing innovate collaborate ma...,"['job', 'description', 'boeing', 'innovate', '..."
4,"Lead, Software Engineer",Allvue Systems,"San Diego, CA","About Allvue\n\nWe are Allvue Systems, the lea...",software engineer,allvue allvue systems leading provider softwar...,"['allvue', 'allvue', 'systems', 'leading', 'pr..."


In [9]:
df_combined['role'].value_counts()

role
accountant           99
software engineer    95
data analyst         91
Name: count, dtype: int64

In [11]:
vectorizer = TfidfVectorizer(max_features = 3000)
X = vectorizer.fit_transform(df_combined['clean_description'])
y = df_combined['role']

## Classification Model and Accuracy

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

lr = LogisticRegression(max_iter = 1000)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9649122807017544

Classification Report:
                    precision    recall  f1-score   support

       accountant       1.00      0.90      0.95        20
     data analyst       1.00      1.00      1.00        18
software engineer       0.90      1.00      0.95        19

         accuracy                           0.96        57
        macro avg       0.97      0.97      0.97        57
     weighted avg       0.97      0.96      0.96        57



## Topic Model (LDA)

In [19]:
lda = LatentDirichletAllocation(n_components = 3, random_state = 42)
fitted = lda.fit_transform(X)

predicted = fitted.argmax(axis = 1)

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(y)

rand_score = adjusted_rand_score(labels, predicted)
print("\nAdjusted Rand Index (topics vs roles):", rand_score)


Adjusted Rand Index (topics vs roles): 0.10695655533075178


In [20]:
#Top 10 words per Topic
feature_names = vectorizer.get_feature_names_out()
for idx, topic in enumerate(lda.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    print(f"\nTopic {idx + 1}: {', '.join(top_words)}")


Topic 1: data, accounting, financial, experience, accountant, work, business, skills, analysis, reporting

Topic 2: original, posting, see, job, skyworks, targeting, bay, authorities, interesting, enables

Topic 3: software, engineering, development, experience, design, embedded, engineer, systems, space, engineers
